# PREDICTING AN OPTIMAL LOCATION FOR A RESTAURANT

This project will try to find a optimal location for opening a restaurant in the city of Calgary,Canada.
This is in benefit of the stake holders looking for an opportunity to open a restaurant.

In [ ]:


























#importing necessary libraries

import requests 
import pandas as pd
import numpy as np 

# module to convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Foursquare API is used to extract all the location data which will be needed in the project.

In [4]:
#foursquare credentials
CLIENT_ID = 'P3Y43JD3OCBZ4WA2RDMU0WJ0GNRZYAANVEDASMEDZOFRWOEU' 
CLIENT_SECRET = 'DK1YSHHS1KOQAMBYFKJHTQ1AH30EJXLTROIXVETQMTGT2TJH' 
VERSION = '20180604'
LIMIT = 100
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [5]:
#getting the latitude and longitude of Downtown, Calgary
address = 'Downtown, Calgary, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

51.0464963 -114.0506433


In [6]:
#searching for all the restaurants 
search_query='Restaurant'
radius=10000
url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,latitude,longitude,VERSION,search_query,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=P3Y43JD3OCBZ4WA2RDMU0WJ0GNRZYAANVEDASMEDZOFRWOEU&client_secret=DK1YSHHS1KOQAMBYFKJHTQ1AH30EJXLTROIXVETQMTGT2TJH&ll=51.0464963,-114.0506433&v=20180604&query=Restaurant&radius=10000&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d51dcb56f0aa20032a509b9'},
 'response': {'venues': [{'id': '4b0586ebf964a520997522e3',
    'name': 'Thomsons Restaurant',
    'location': {'address': '112 Stephen Avenue Walk',
     'lat': 51.04559333495997,
     'lng': -114.0626634601612,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.04559333495997,
       'lng': -114.0626634601612}],
     'distance': 847,
     'postalCode': 'T2G 5P6',
     'cc': 'CA',
     'city': 'Calgary',
     'state': 'AB',
     'country': 'Canada',
     'formattedAddress': ['112 Stephen Avenue Walk',
      'Calgary AB T2G 5P6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1565646006',
    'hasPerk': False},
   {'id': '4b05

In [11]:
venues=results['response']['venues']
dataframe=json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4b0586ebf964a520997522e3,112 Stephen Avenue Walk,CA,Calgary,Canada,NaN,847,"[112 Stephen Avenue Walk, Calgary AB T2G 5P6, ...","[{'label': 'display', 'lat': 51.04559333495997...",51.045593,-114.062663,NaN,T2G 5P6,AB,Thomsons Restaurant,v-1565646006,NaN
1,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4b0586e7f964a520fc7322e3,4809 Centre St. N.W.,CA,Calgary,Canada,NaN,838,"[4809 Centre St. N.W., Calgary AB T2E 2Z6, Can...","[{'label': 'display', 'lat': 51.04822577360245...",51.048226,-114.062300,NaN,T2E 2Z6,AB,Dons Restaurant,v-1565646006,NaN
2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4cefec2df98ba090c767c673,"102, 550 6th Ave SW",CA,Calgary,Canada,In Calgary House,1600,"[102, 550 6th Ave SW (In Calgary House), Calga...","[{'label': 'display', 'lat': 51.0475801543888,...",51.047580,-114.073445,NaN,T2P 0S2,AB,Green Bean Restaurant & Catering,v-1565646006,50559442
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b36549af964a520fd3225e3,"#6, 1423 Centre St. NW",CA,Calgary,Canada,Centre Street N & 14th Ave NW,2232,"[#6, 1423 Centre St. NW (Centre Street N & 14t...","[{'label': 'display', 'lat': 51.06504908078969...",51.065049,-114.062747,NaN,T2E 2R9,AB,Sun's BBQ Restaurant,v-1565646006,NaN
4,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",False,4b0586e8f964a5205b7422e3,309 2nd Ave. S.W.,CA,Calgary,Canada,NaN,1297,"[309 2nd Ave. S.W., Calgary AB T2P 0C5, Canada]","[{'label': 'display', 'lat': 51.051163, 'lng':...",51.051163,-114.067639,NaN,T2P 0C5,AB,Subway Restaurant,v-1565646006,NaN


In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Thomsons Restaurant,American Restaurant,112 Stephen Avenue Walk,CA,Calgary,Canada,NaN,847,"[112 Stephen Avenue Walk, Calgary AB T2G 5P6, ...","[{'label': 'display', 'lat': 51.04559333495997...",51.045593,-114.062663,NaN,T2G 5P6,AB,4b0586ebf964a520997522e3
1,Dons Restaurant,Bar,4809 Centre St. N.W.,CA,Calgary,Canada,NaN,838,"[4809 Centre St. N.W., Calgary AB T2E 2Z6, Can...","[{'label': 'display', 'lat': 51.04822577360245...",51.048226,-114.062300,NaN,T2E 2Z6,AB,4b0586e7f964a520fc7322e3
2,Green Bean Restaurant & Catering,Restaurant,"102, 550 6th Ave SW",CA,Calgary,Canada,In Calgary House,1600,"[102, 550 6th Ave SW (In Calgary House), Calga...","[{'label': 'display', 'lat': 51.0475801543888,...",51.047580,-114.073445,NaN,T2P 0S2,AB,4cefec2df98ba090c767c673
3,Sun's BBQ Restaurant,Chinese Restaurant,"#6, 1423 Centre St. NW",CA,Calgary,Canada,Centre Street N & 14th Ave NW,2232,"[#6, 1423 Centre St. NW (Centre Street N & 14t...","[{'label': 'display', 'lat': 51.06504908078969...",51.065049,-114.062747,NaN,T2E 2R9,AB,4b36549af964a520fd3225e3
4,Subway Restaurant,Cocktail Bar,309 2nd Ave. S.W.,CA,Calgary,Canada,NaN,1297,"[309 2nd Ave. S.W., Calgary AB T2P 0C5, Canada]","[{'label': 'display', 'lat': 51.051163, 'lng':...",51.051163,-114.067639,NaN,T2P 0C5,AB,4b0586e8f964a5205b7422e3
5,Añejo Restaurant,Mexican Restaurant,#2 2116 4 St SW,CA,Calgary,Canada,22 Ave SW,2029,"[#2 2116 4 St SW (22 Ave SW), Calgary AB T2S 1...","[{'label': 'display', 'lat': 51.03373287315315...",51.033733,-114.071343,NaN,T2S 1W7,AB,504a96a0e4b006c4356d5036
6,Phoenix Express Vietnamese Restaurant,Vietnamese Restaurant,"# 200, 500 - 4 Avenue S.W.",CA,Calgary,Canada,NaN,1518,"[# 200, 500 - 4 Avenue S.W., Calgary AB T2P 2V...","[{'label': 'display', 'lat': 51.04987177934046...",51.049872,-114.071665,NaN,T2P 2V6,AB,4bd72d4f35aad13a2f518ef3
7,The Deane House Restaurant,Restaurant,806 9 Ave. Southeast,CA,Calgary,Canada,NaN,682,"[806 9 Ave. Southeast, Calgary AB T2G 5E1, Can...","[{'label': 'display', 'lat': 51.04373092981922...",51.043731,-114.041941,NaN,T2G 5E1,AB,4b0586eaf964a520317522e3
8,Glass Restaurant & Lounge,Restaurant,815 7th Avenue SW,CA,Calgary,Canada,NaN,2040,"[815 7th Avenue SW, Calgary AB T2P 1A2, Canada]","[{'label': 'display', 'lat': 51.04667076509864...",51.046671,-114.079797,NaN,T2P 1A2,AB,4bc36563920eb71356641d2c
9,Calgary Court Restaurant,Chinese Restaurant,119 - 2 Ave SE,CA,Calgary,Canada,NaN,915,"[119 - 2 Ave SE, Calgary AB T2G 0B2, Canada]","[{'label': 'display', 'lat': 51.051127974677, ...",51.051128,-114.061454,NaN,T2G 0B2,AB,4b6b6ebef964a52070082ce3


In [13]:
#df_k=dataframe_filtered.drop(['id','state','postalCode','neighborhood'],axis=1)
df_k=dataframe_filtered.dropna(subset=['address'])
df_final=df_k[['distance','lat','lng']]
#dataframe_filtered.dropna(subset=['address'])
#dataframe_filtered.shape

In [14]:
cal_map=folium.Map(location=[latitude,longitude],zoom_start=13)
a=folium.map.FeatureGroup()
a.add_child(
    folium.CircleMarker(
    [latitude,longitude],
    radius=10,
    color='red',
    popup='Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6))
cal_map.add_child(a)

for lat, lng, in zip(dataframe_filtered.lat, dataframe_filtered.lng):
    a.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6))   
    
cal_map.add_child(a)
cal_map

In [15]:
X=df_final.values[:,1:]
X=np.nan_to_num(X)
Clus_df=StandardScaler().fit_transform(X)
Clus_df

array([[-0.03959382, -0.01521019],
       [ 0.17289702,  0.01569707],
       [ 0.12078254, -0.93141339],
       [ 1.53087684, -0.02234722],
       [ 0.40999036, -0.43804401],
       [-0.99697204, -0.75282668],
       [ 0.30576284, -0.7801918 ],
       [-0.18992743,  1.74583984],
       [ 0.0473765 , -1.4712462 ],
       [ 0.40716311,  0.08760755],
       [ 1.69559361, -0.5261812 ],
       [ 0.07653544,  0.35560756],
       [ 0.26254801,  0.01566499],
       [ 0.3615427 , -0.02020253],
       [ 0.40833922,  0.10560241],
       [-0.04923265,  0.13206327],
       [ 0.56169679, -1.9801383 ],
       [-0.66255427, -1.35874906],
       [ 1.38105404, -2.28770554],
       [ 0.56658745,  1.73436363],
       [ 0.37751366,  0.1794014 ],
       [ 0.35757267, -0.06640783],
       [ 0.27145603,  0.09543381],
       [-0.0418206 , -0.10951854],
       [-0.91382302,  1.73521115],
       [-0.39361499,  0.26882778],
       [ 0.17328781,  0.05064903],
       [-0.42525341,  2.500708  ],
       [ 0.25210176,

In [16]:
clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0
 0 2 0 1 0 0]


In [17]:
df_k["Clus_km"] = labels
df_k

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Clus_km
0,Thomsons Restaurant,American Restaurant,112 Stephen Avenue Walk,CA,Calgary,Canada,NaN,847,"[112 Stephen Avenue Walk, Calgary AB T2G 5P6, ...","[{'label': 'display', 'lat': 51.04559333495997...",51.045593,-114.062663,NaN,T2G 5P6,AB,4b0586ebf964a520997522e3,0
1,Dons Restaurant,Bar,4809 Centre St. N.W.,CA,Calgary,Canada,NaN,838,"[4809 Centre St. N.W., Calgary AB T2E 2Z6, Can...","[{'label': 'display', 'lat': 51.04822577360245...",51.048226,-114.062300,NaN,T2E 2Z6,AB,4b0586e7f964a520fc7322e3,0
2,Green Bean Restaurant & Catering,Restaurant,"102, 550 6th Ave SW",CA,Calgary,Canada,In Calgary House,1600,"[102, 550 6th Ave SW (In Calgary House), Calga...","[{'label': 'display', 'lat': 51.0475801543888,...",51.047580,-114.073445,NaN,T2P 0S2,AB,4cefec2df98ba090c767c673,0
3,Sun's BBQ Restaurant,Chinese Restaurant,"#6, 1423 Centre St. NW",CA,Calgary,Canada,Centre Street N & 14th Ave NW,2232,"[#6, 1423 Centre St. NW (Centre Street N & 14t...","[{'label': 'display', 'lat': 51.06504908078969...",51.065049,-114.062747,NaN,T2E 2R9,AB,4b36549af964a520fd3225e3,0
4,Subway Restaurant,Cocktail Bar,309 2nd Ave. S.W.,CA,Calgary,Canada,NaN,1297,"[309 2nd Ave. S.W., Calgary AB T2P 0C5, Canada]","[{'label': 'display', 'lat': 51.051163, 'lng':...",51.051163,-114.067639,NaN,T2P 0C5,AB,4b0586e8f964a5205b7422e3,0
5,Añejo Restaurant,Mexican Restaurant,#2 2116 4 St SW,CA,Calgary,Canada,22 Ave SW,2029,"[#2 2116 4 St SW (22 Ave SW), Calgary AB T2S 1...","[{'label': 'display', 'lat': 51.03373287315315...",51.033733,-114.071343,NaN,T2S 1W7,AB,504a96a0e4b006c4356d5036,0
6,Phoenix Express Vietnamese Restaurant,Vietnamese Restaurant,"# 200, 500 - 4 Avenue S.W.",CA,Calgary,Canada,NaN,1518,"[# 200, 500 - 4 Avenue S.W., Calgary AB T2P 2V...","[{'label': 'display', 'lat': 51.04987177934046...",51.049872,-114.071665,NaN,T2P 2V6,AB,4bd72d4f35aad13a2f518ef3,0
7,The Deane House Restaurant,Restaurant,806 9 Ave. Southeast,CA,Calgary,Canada,NaN,682,"[806 9 Ave. Southeast, Calgary AB T2G 5E1, Can...","[{'label': 'display', 'lat': 51.04373092981922...",51.043731,-114.041941,NaN,T2G 5E1,AB,4b0586eaf964a520317522e3,2
8,Glass Restaurant & Lounge,Restaurant,815 7th Avenue SW,CA,Calgary,Canada,NaN,2040,"[815 7th Avenue SW, Calgary AB T2P 1A2, Canada]","[{'label': 'display', 'lat': 51.04667076509864...",51.046671,-114.079797,NaN,T2P 1A2,AB,4bc36563920eb71356641d2c,0
9,Calgary Court Restaurant,Chinese Restaurant,119 - 2 Ave SE,CA,Calgary,Canada,NaN,915,"[119 - 2 Ave SE, Calgary AB T2G 0B2, Canada]","[{'label': 'display', 'lat': 51.051127974677, ...",51.051128,-114.061454,NaN,T2G 0B2,AB,4b6b6ebef964a52070082ce3,0


In [18]:
df_k.groupby('Clus_km').mean()

,distance,lat,lng
Clus_km,,,
0,1313.72973,51.048315,-114.066307
1,7315.00000,50.980992,-114.042328
2,1182.40000,51.042591,-114.038232


In [19]:


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i + x + (i*x)**2 for i in range(clusterNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_k['lat'], df_k['lng'], df_k['name'], df_k['Clus_km']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
df_k.loc[df_k['Clus_km'] == 0, df_k.columns[[1] + list(range(5, df_k.shape[1]))]]

,categories,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Clus_km
0,American Restaurant,Canada,NaN,847,"[112 Stephen Avenue Walk, Calgary AB T2G 5P6, ...","[{'label': 'display', 'lat': 51.04559333495997...",51.045593,-114.062663,NaN,T2G 5P6,AB,4b0586ebf964a520997522e3,0
1,Bar,Canada,NaN,838,"[4809 Centre St. N.W., Calgary AB T2E 2Z6, Can...","[{'label': 'display', 'lat': 51.04822577360245...",51.048226,-114.062300,NaN,T2E 2Z6,AB,4b0586e7f964a520fc7322e3,0
2,Restaurant,Canada,In Calgary House,1600,"[102, 550 6th Ave SW (In Calgary House), Calga...","[{'label': 'display', 'lat': 51.0475801543888,...",51.047580,-114.073445,NaN,T2P 0S2,AB,4cefec2df98ba090c767c673,0
3,Chinese Restaurant,Canada,Centre Street N & 14th Ave NW,2232,"[#6, 1423 Centre St. NW (Centre Street N & 14t...","[{'label': 'display', 'lat': 51.06504908078969...",51.065049,-114.062747,NaN,T2E 2R9,AB,4b36549af964a520fd3225e3,0
4,Cocktail Bar,Canada,NaN,1297,"[309 2nd Ave. S.W., Calgary AB T2P 0C5, Canada]","[{'label': 'display', 'lat': 51.051163, 'lng':...",51.051163,-114.067639,NaN,T2P 0C5,AB,4b0586e8f964a5205b7422e3,0
5,Mexican Restaurant,Canada,22 Ave SW,2029,"[#2 2116 4 St SW (22 Ave SW), Calgary AB T2S 1...","[{'label': 'display', 'lat': 51.03373287315315...",51.033733,-114.071343,NaN,T2S 1W7,AB,504a96a0e4b006c4356d5036,0
6,Vietnamese Restaurant,Canada,NaN,1518,"[# 200, 500 - 4 Avenue S.W., Calgary AB T2P 2V...","[{'label': 'display', 'lat': 51.04987177934046...",51.049872,-114.071665,NaN,T2P 2V6,AB,4bd72d4f35aad13a2f518ef3,0
8,Restaurant,Canada,NaN,2040,"[815 7th Avenue SW, Calgary AB T2P 1A2, Canada]","[{'label': 'display', 'lat': 51.04667076509864...",51.046671,-114.079797,NaN,T2P 1A2,AB,4bc36563920eb71356641d2c,0
9,Chinese Restaurant,Canada,NaN,915,"[119 - 2 Ave SE, Calgary AB T2G 0B2, Canada]","[{'label': 'display', 'lat': 51.051127974677, ...",51.051128,-114.061454,NaN,T2G 0B2,AB,4b6b6ebef964a52070082ce3,0
10,Chinese Restaurant,Canada,at 2 St.NW,2616,"[320 16 Ave NW (at 2 St.NW), Calgary AB, Canada]","[{'label': 'display', 'lat': 51.0670896712542,...",51.067090,-114.068676,NaN,NaN,AB,4bdd0ab98f53d13a65a9eb8f,0


In [24]:
df_k.loc[df_k['Clus_km'] == 1, df_k.columns[[1] + list(range(5, df_k.shape[1]))]]

,categories,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Clus_km
46,Café,Canada,NaN,7315,"[8000 11 St. SE, Calgary AB T2H, Canada]","[{'label': 'display', 'lat': 50.98099246819835...",50.980992,-114.042328,NaN,T2H,AB,4ebef8b6f5b93923c12750e3,1


In [25]:
df_k.loc[df_k['Clus_km'] == 2, df_k.columns[[1] + list(range(5, df_k.shape[1]))]]

,categories,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Clus_km
7,Restaurant,Canada,NaN,682,"[806 9 Ave. Southeast, Calgary AB T2G 5E1, Can...","[{'label': 'display', 'lat': 51.04373092981922...",51.043731,-114.041941,NaN,T2G 5E1,AB,4b0586eaf964a520317522e3,2
22,Vietnamese Restaurant,Canada,NaN,948,"[920 - 1 Avenue NE, Calgary AB T2E, Canada]","[{'label': 'display', 'lat': 51.053103, 'lng':...",51.053103,-114.042076,NaN,T2E,AB,5b4ff8289411f2002cd90032,2
28,Restaurant,Canada,NaN,1437,"[2016 spiller road se, Calgary AB, Canada]","[{'label': 'display', 'lat': 51.03476296321584...",51.034763,-114.042066,NaN,NaN,AB,4e1ce56e14955b99c4122a63,2
31,Vietnamese Restaurant,Canada,9 Avenue Se,1383,"[1318 9 Ave. SE (9 Avenue Se), Calgary AB T2G ...","[{'label': 'display', 'lat': 51.0408155985169,...",51.040816,-114.033058,NaN,T2G 0T3,AB,4b08bdedf964a520b21123e3,2
44,Pub,Canada,btwn 12th & 13th St.,1462,"[1336 9 Ave. SE (btwn 12th & 13th St.), Calgar...","[{'label': 'display', 'lat': 51.04054168704647...",51.040542,-114.032021,NaN,NaN,AB,4b0eed1af964a520e25c23e3,2
